In [476]:
import pandas as pd
import re

# Read in xlsx feature dataset (for symptoms data)
dfs = pd.read_excel('GSE198449_PCR_and_Symptoms_Full_Info.xlsx', sheet_name=None)
dfs = dfs["Sheet1"]

# Subsample dataset to standardize time for people tested at T28
dfs["Unnamed: 3"] = dfs["Unnamed: 3"] == "28"
dfs = dfs.loc[dfs["Unnamed: 3"], :]

# Read in counts/expression dataset
dfs2 = pd.read_csv('Exprs.csv')

# Subsample for T28 and keep the index/Symbol column
dfs2 = dfs2.loc[:,(dfs2.columns.str[10:12] == "28") | (dfs2.columns.str[0] == "U") | (dfs2.columns.str[0] == "S")]
dfs2.head(2)

,Unnamed: 0,SYMBOL,X20_6537.T28_P2R,X20_5063.T28_P4,X20_5134.T28_P4,X20_5326.T28_P4,X20_5411.T28_P4,X20_5420.T28_P4,X20_5568.T28_P4,X20_5615.T28_P4,...,X20_9634.T28_P13,X20_9424.T28_P14,X20_7690.T28_P15,X20_8167.T28_P16,X20_8263.T28_P17,X20_5300.T28_P20,X20_6228.T28_P21,X20_5719.T28_P22,X20_7647.T28_P25,X20_9571.T28_P25
0,1,TSPAN6,NaN,2.321928,1.0,NaN,1.584963,2.0,0.0,1.584963,...,NaN,3.169925,3.0,2.321928,0.0,1.584963,2.807355,1.0,NaN,2.0
1,2,TNMD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [465]:
# Rename counts dataset columns to remove redundant data in titles
dtcol = [col for col in dfs2.columns if "X" in col]
for ex in dtcol:
   dfs2.rename(columns={ex: ex[1:8] + "-T28"}, inplace=True)
dfs2.head(2)

,Unnamed: 0,SYMBOL,20_6537-T28,20_5063-T28,20_5134-T28,20_5326-T28,20_5411-T28,20_5420-T28,20_5568-T28,20_5615-T28,...,20_9634-T28,20_9424-T28,20_7690-T28,20_8167-T28,20_8263-T28,20_5300-T28,20_6228-T28,20_5719-T28,20_7647-T28,20_9571-T28
0,1,TSPAN6,NaN,2.321928,1.0,NaN,1.584963,2.0,0.0,1.584963,...,NaN,3.169925,3.0,2.321928,0.0,1.584963,2.807355,1.0,NaN,2.0
1,2,TNMD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [466]:
## Feature dataset reformatting ##

# Remove irrelevant columns of data
dfs.drop("Unnamed: 4", axis=1)
dfs = dfs.iloc[: , 1:]
del dfs['Unnamed: 2']
del dfs['Unnamed: 3']
del dfs['Unnamed: 4']

# Rename column titles in feature dataset
dfs = dfs.rename(columns={'Unnamed: 1':'PatientID', 'Unnamed: 5':'Symptoms'})
# reorientate index column
dfs = dfs.reset_index(drop=True)

# Convert symptom column to True/False values (groups)
dfs['Symptoms'] = dfs['Symptoms'].astype(str)
dfs["Symptoms"] = dfs["Symptoms"] != "nan"
dfs.shape

(408, 2)

In [467]:
# Remove duplicate columns in gene datasets
dfs2_hash = {}
non_unique = set()
for x in dfs2.columns:
    if x not in dfs2_hash:
        dfs2_hash[x] = 0
    dfs2_hash[x] +=1
    if dfs2_hash[x] > 1:
        non_unique.add(x)
for x in non_unique:
    dfs2 = dfs2.drop(x, axis=1)

# create dictionary for each p in our gene dataset so we can drop extra tables from our feature dataset
dict = {}
for col in dfs2.columns:
    if col not in dict:
        dict[col] = 1

# drop feature rows not found in gene dataset (make both datasets same size/sample data)
for i, row in enumerate(dfs.values.tolist()):
    if row[0] not in dict:
        dfs = dfs.drop(labels=i, axis=0)

print("NEW EXPRESSION SHAPE", dfs2.shape)
print("NEW FEATURE SHAPE", dfs.shape)

# Note: should be 2 columns more since index and symbol column

NEW EXPRESSION SHAPE (35537, 151)
NEW FEATURE SHAPE (149, 2)


In [468]:
# remove index column
del dfs2['Unnamed: 0']
# sort p id's columns
dfs2 = dfs2.reindex(sorted(dfs2.columns), axis=1)

# move symbol back to 1st col
column_to_move = dfs2.pop("SYMBOL")
dfs2.insert(0, "SYMBOL", column_to_move)

# sort rows in feature dataset by id
dfs = dfs.sort_values(by=['PatientID'])
# reset index col
dfs = dfs.reset_index(drop=True)
dfs.head(2)

,PatientID,Symptoms
0,20_5011-T28,True
1,20_5063-T28,True


In [471]:
# remove duplicate gene rows
dfs2 = dfs2.drop_duplicates(subset=['SYMBOL'])

# Reformat patient id to match gene cols
dfs['PatientID'] = 'X' + dfs['PatientID'].astype(str)
dfs['PatientID'] = dfs['PatientID'].str[:-4] + ".T28"


In [341]:
dfs2.to_csv("e_data.csv",index=False)
dfs.to_csv("f_data.csv",index=False)

(35292, 150)